In [1]:

import json
import shutil
import os
import pickle
from callback import MultipleClassAUROC, MultiGPUModelCheckpoint
from configparser import ConfigParser
from generator import AugmentedImageSequence
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import multi_gpu_model
from utility import get_sample_counts
from weights import get_class_weights
from augmenter import augmenter
from tensorflow.keras import backend as K
import pandas as pd
import numpy as np
import time
import tensorflow.keras.initializers
import statistics
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer, Flatten, Input, GaussianNoise
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras_radam import RAdam
from sklearn.metrics import confusion_matrix

%load_ext tensorboard

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

import numpy as np

#print("TensorFlow version: ", tf.__version__)
#assert version.parse(tf.__version__).release[0] >= 2, \
#    "This notebook requires TensorFlow 2.0 or above."


Using TensorFlow backend.


In [2]:
import gc

In [3]:
import random 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
# UPDATED: import from tensorflow.keras instead of keras
from tensorflow.keras import layers, optimizers, losses, metrics
import gc
import tensorflow_addons as tfa

In [4]:
oneClass = "Effusion"

In [5]:
config_file = "./config.ini"
cp = ConfigParser()
cp.read(config_file)

    # default config
output_dir = cp["DEFAULT"].get("output_dir")
image_source_dir = cp["DEFAULT"].get("image_source_dir")
base_model_name = cp["DEFAULT"].get("base_model_name")
class_names = cp["DEFAULT"].get("class_names").split(",")

    # train config
use_base_model_weights = cp["TRAIN"].getboolean("use_base_model_weights")
use_trained_model_weights = cp["TRAIN"].getboolean("use_trained_model_weights")
use_best_weights = cp["TRAIN"].getboolean("use_best_weights")
output_weights_name = cp["TRAIN"].get("output_weights_name")
epochs = cp["TRAIN"].getint("epochs")
batch_size = cp["TRAIN"].getint("batch_size")
initial_learning_rate = cp["TRAIN"].getfloat("initial_learning_rate")
generator_workers = cp["TRAIN"].getint("generator_workers")
image_dimension = cp["TRAIN"].getint("image_dimension")
train_steps = cp["TRAIN"].get("train_steps")
patience_reduce_lr = cp["TRAIN"].getint("patience_reduce_lr")
min_lr = cp["TRAIN"].getfloat("min_lr")
validation_steps = cp["TRAIN"].get("validation_steps")
positive_weights_multiply = cp["TRAIN"].getfloat("positive_weights_multiply")
dataset_csv_dir = cp["TRAIN"].get("dataset_csv_dir")

In [6]:
def focal_loss(gamma=1.0, alpha=0.5):
    gamma = float(gamma)
    alpha = float(alpha)
    def focal_loss_fixed(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0-epsilon)
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [7]:
from tensorflow.keras.losses import Huber

In [8]:
def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(tf.cast(y_pred,tf.float32) - tf.cast(y_true,tf.float32)), axis=-1))

In [9]:
def construct_network1(dropout=0.08425517073874295, neuronPct=0.1767547775828121, neuronShrink=0.33180474398878285):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat1'))
    model.add(Dense(neuronCount,name='dense1'))
    model.add(Activation('relu',name='relu1'))
    model.add(Dropout(dropout, name='dropout1'))
    model.add(Dense(14, activation='sigmoid',name='midLayer1')) # Output
    weights_path = None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer1')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer1')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [10]:
def construct_network2(dropout=0.15672137551441198, neuronPct=0.2197894476507525, neuronShrink=0.3803316528497302, noisePct=0.282563134185142):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat2'))
    model.add(Dense(neuronCount,name='dense2'))
    model.add(GaussianNoise(noisePct))
    model.add(Activation('relu',name='relu2'))
    model.add(Dropout(dropout, name='dropout2'))
    model.add(Dense(14, activation='sigmoid',name='midLayer2')) # Output
    weights_path= None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    #model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer2')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer2')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [11]:
def construct_network(model1,model2):
    model = Model([model1.input,model2.input], [model1.output,model2.output])
    return model 

In [12]:
# compute steps
train_counts, train_pos_counts = get_sample_counts(output_dir, "train"+oneClass, class_names)
dev_counts, _ = get_sample_counts(output_dir, "dev"+oneClass, class_names)
    
if train_steps == "auto":
    train_steps = int(train_counts / batch_size)
else:
    try:
        train_steps = int(train_steps)
    except ValueError:
        raise ValueError(f"""train_steps: {train_steps} is invalid,please use 'auto' or integer.""")
    print(f"** train_steps: {train_steps} **")

if validation_steps == "auto":
    validation_steps = int(dev_counts / batch_size)
else:
    try:
        validation_steps = int(validation_steps)
    except ValueError:
        raise ValueError(f"""validation_steps: {validation_steps} is invalid,please use 'auto' or integer.""")
        print(f"** validation_steps: {validation_steps} **")

        # compute class weights
keras.backend.clear_session()
print("** compute class weights from training data **")
class_weights = get_class_weights(train_counts,train_pos_counts,multiply=positive_weights_multiply,)
print("** class_weights **")
print(class_weights)
#print(str(train_steps))
#print(str(train_counts))
#print(str(batch_size))

** compute class weights from training data **
2226: 9261
741: 9261
9261: 9261
2801: 9261
865: 9261
645: 9261
195: 9261
711: 9261
896: 9261
437: 9261
265: 9261
126: 9261
594: 9261
9: 9261
** class_weights **
[{0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}]


In [13]:
test_steps = cp["TEST"].get("test_steps")
test_counts, _ = get_sample_counts(output_dir, "test", class_names)

if test_steps == "auto":
    test_steps = int(test_counts / batch_size)
else:
    try:
        test_steps = int(test_steps)
    except ValueError:
        raise ValueError(f"""test_steps: {test_steps} is invalid,please use 'auto' or integer.""")
        
print(f"** test_steps: {test_steps} **")

** test_steps: 22433 **


In [14]:
train_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "train"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=train_steps,
        )
validation_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "dev"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=validation_steps,
            shuffle_on_epoch_end=False,
)

test_sequence = AugmentedImageSequence(
        dataset_csv_file=os.path.join(output_dir, "test.csv"),
        class_names=class_names,
        source_image_dir=image_source_dir,
        batch_size=batch_size,
        target_size=(image_dimension, image_dimension),
        augmenter=None,
        steps=test_steps,
        shuffle_on_epoch_end=False,
    )

In [15]:
def optimize_network(lr):
    gc.collect()
      # Define the Keras TensorBoard callback.
    logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

    start_time = time.time()
    model1 = construct_network1()
    model2 = construct_network2()
    
    optimizer = SGD(lr=initial_learning_rate)
    
    alpha = 0.9340456763831478
    gamma = 1.4195808780694898
    model1.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss})

    alpha = 0.7297456293468533
    gamma = 1.2700405014991505
    model2.compile(optimizer=optimizer,loss={'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})
  
    model = construct_network(model1=model1,model2=model2)
    model.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss,'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})

    output_weights_path = os.path.join(output_dir,  str(lr)+"_"+output_weights_name)
    
    print(f"** set output weights path to: {output_weights_path} **")
                  
    
                  
    checkpoint = ModelCheckpoint(
                 output_weights_path,
                 save_weights_only=True,
                 save_best_only=True,
                 verbose=1,
            )
    start_time = time.time()
  
    model.summary()
  
    callbacks = [
            checkpoint,
            #keras.callbacks.TensorBoard(log_dir=logdir),
            #TensorBoard(log_dir=os.path.join(output_dir, "logs"), batch_size=batch_size),
            ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=patience_reduce_lr,
                              verbose=1, mode="min", min_lr=min_lr), 
            EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto', restore_best_weights=True)
    ]
    
    
    history = model.fit_generator(
            generator=train_sequence,
            steps_per_epoch=train_steps,
            epochs=epochs,
            validation_data=validation_sequence,
            validation_steps=validation_steps,
            callbacks=callbacks,
            class_weight=[class_weights,class_weights,class_weights,class_weights,class_weights,class_weights],
            workers=generator_workers,
            shuffle=False,
        )
        
    y_hat = model.predict_generator(test_sequence, verbose=1)
    y = test_sequence.get_y_true()
    
    test_log_path = os.path.join(output_dir, str(lr)+"_"+"test.log")
    print(f"** write log to {test_log_path} **")
    aurocs = []
    auprcs = []
    precision = dict()
    recall = dict()
    threshold = dict()
    with open(test_log_path, "w") as f:
        for k in range(6):
            for i in range(len(class_names)):
                 if(class_names[i] == str(oneClass)):
                
                    try:
                        score = roc_auc_score(y[:, i], y_hat[k][:, i])
                        precision[i], recall[i], threshold[i] = precision_recall_curve(y[:, i], y_hat[k][:, i])
                        tmp = auc(recall[i], precision[i])
                        aurocs.append(score)
                        auprcs.append(tmp) 
                    except ValueError:
                        score = 0
               
                    print(f"auroc {str(k)+class_names[i]}: {score}\n")
                    print(f"auprc {str(k)+class_names[i]}: {tmp}\n")
                    f.write(f"auroc {str(k)+class_names[i]}: {score}\n")
                    f.write(f"auprc {str(k)+class_names[i]}: {tmp}\n")
        
        mean_auroc = np.mean(aurocs)
        mean_auprc = float(np.mean(auprcs))
        f.write("-------------------------\n")
        f.write(f"mean auroc: {mean_auroc}\n")
        print(f"mean auroc: {mean_auroc}\n")
        f.write(f"mean auprc: {mean_auprc}\n")
        print(f"mean auprc: {mean_auprc}\n")
        
        max_auroc = np.max(aurocs)
        max_auprc = float(np.max(auprcs))
        f.write("-------------------------\n")
        f.write(f"max auroc: {max_auroc}\n")
        print(f"max auroc: {max_auroc}\n")
        f.write(f"max auprc: {max_auprc}\n")
        print(f"max auprc: {max_auprc}\n")
    
    keras.backend.clear_session()
    time_took = time.time() - start_time
    
    return max_auroc, time_took
    


In [16]:
step = np.arange(0.009, 0.0151, 0.001)
maxi = []
for k in np.nditer(step):
    opn, daTime = optimize_network(k)
    print(daTime)
    maxi.append(opn)




** set output weights path to: ./experiments/0.009_weights.h5 **
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
flat2 (Flatten)                 (None, 1536)         0           input_2[0][0]                    
__________________________________________________________________________________________________
flat1 (Flatten)                 (None, 1536)         0           input_1[0][0]                    
___________________________

Epoch 6/11
9240/9261 [============================>.] - ETA: 0s - loss: 4.6383 - leftLayer1_loss: 0.0615 - midLayer1_loss: 1.3414 - rightLayer1_loss: 0.8914 - leftLayer2_loss: 0.0425 - midLayer2_loss: 1.4301 - rightLayer2_loss: 0.8715
Epoch 00006: val_loss improved from 4.62827 to 4.60214, saving model to ./experiments/0.009_weights.h5
9261/9261 [==============================] - 24s 3ms/step - loss: 4.6379 - leftLayer1_loss: 0.0615 - midLayer1_loss: 1.3414 - rightLayer1_loss: 0.8912 - leftLayer2_loss: 0.0425 - midLayer2_loss: 1.4300 - rightLayer2_loss: 0.8713 - val_loss: 4.6021 - val_leftLayer1_loss: 0.0591 - val_midLayer1_loss: 1.3331 - val_rightLayer1_loss: 0.8814 - val_leftLayer2_loss: 0.0566 - val_midLayer2_loss: 1.3388 - val_rightLayer2_loss: 0.9332
Epoch 7/11
9252/9261 [============================>.] - ETA: 0s - loss: 4.6220 - leftLayer1_loss: 0.0573 - midLayer1_loss: 1.3412 - rightLayer1_loss: 0.8841 - leftLayer2_loss: 0.0407 - midLayer2_loss: 1.4286 - rightLayer2_loss: 0.8701

  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 9261 steps, validate for 1292 steps
Epoch 1/11
9253/9261 [============================>.] - ETA: 0s - loss: 5.6640 - leftLayer1_loss: 0.1127 - midLayer1_loss: 1.4261 - rightLayer1_loss: 1.4110 - leftLayer2_loss: 0.1013 - midLayer2_loss: 1.4653 - rightLayer2_loss: 1.1477
Epoch 00001: val_loss improved from inf to 5.25270, saving model to ./experiments/0.009999999999999998_weights.h5
9261/9261 [==============================] - 24s 3ms/step - loss: 5.6635 - leftLayer1_loss: 0.1126 - midLayer1_loss: 1.4261 - rightLayer1_loss: 1.4108 - leftLayer2_loss: 0.1013 - midLayer2_loss: 1.4653 - rightLayer2_loss: 1.1475 - val_loss: 5.2527 - val_leftLayer1_loss: 0.1031 - val_midLayer1_loss: 1.4216 - val_rightLayer1_loss: 1.1240 - val_leftLayer2_loss: 0.0935 - val_midLayer2_loss: 1.4056 - val_rightLayer2_loss: 1

9244/9261 [============================>.] - ETA: 0s - loss: 4.7078 - leftLayer1_loss: 0.0461 - midLayer1_loss: 1.4268 - rightLayer1_loss: 0.8707 - leftLayer2_loss: 0.0368 - midLayer2_loss: 1.4626 - rightLayer2_loss: 0.8647
Epoch 00011: val_loss improved from 4.69970 to 4.69208, saving model to ./experiments/0.009999999999999998_weights.h5
22433/22433 [==============================] - 28s 1ms/step
** write log to ./experiments/0.009999999999999998_test.log **
auroc 0Effusion: 0.48737823032806543

auprc 0Effusion: 0.10955458332938339

auroc 1Effusion: 0.7641201376500975

auprc 1Effusion: 0.2579185623350266

auroc 2Effusion: 0.5294849159068408

auprc 2Effusion: 0.11925661483152925

auroc 3Effusion: 0.4416261073748552

auprc 3Effusion: 0.10190729650956838

auroc 4Effusion: 0.5900382142833287

auprc 4Effusion: 0.16021021037980054

auroc 5Effusion: 0.5744038310895685

auprc 5Effusion: 0.14029375922174245

mean auroc: 0.564508572772126

mean auprc: 0.1481901711011751

max auroc: 0.764120137

Epoch 4/11
9259/9261 [============================>.] - ETA: 0s - loss: 4.6442 - leftLayer1_loss: 0.0705 - midLayer1_loss: 1.3535 - rightLayer1_loss: 0.9076 - leftLayer2_loss: 0.0490 - midLayer2_loss: 1.3845 - rightLayer2_loss: 0.8790
Epoch 00004: val_loss improved from 4.72193 to 4.66680, saving model to ./experiments/0.010999999999999998_weights.h5
9261/9261 [==============================] - 23s 2ms/step - loss: 4.6443 - leftLayer1_loss: 0.0705 - midLayer1_loss: 1.3535 - rightLayer1_loss: 0.9076 - leftLayer2_loss: 0.0490 - midLayer2_loss: 1.3846 - rightLayer2_loss: 0.8790 - val_loss: 4.6668 - val_leftLayer1_loss: 0.0664 - val_midLayer1_loss: 1.3470 - val_rightLayer1_loss: 0.8934 - val_leftLayer2_loss: 0.0641 - val_midLayer2_loss: 1.3327 - val_rightLayer2_loss: 0.9633
Epoch 5/11
9246/9261 [============================>.] - ETA: 0s - loss: 4.6108 - leftLayer1_loss: 0.0632 - midLayer1_loss: 1.3542 - rightLayer1_loss: 0.8942 - leftLayer2_loss: 0.0447 - midLayer2_loss: 1.3801 - rightLaye

...

9259/9261 [============================>.] - ETA: 0s - loss: 4.6582 - leftLayer1_loss: 0.0459 - midLayer1_loss: 1.3694 - rightLayer1_loss: 0.8700 - leftLayer2_loss: 0.0372 - midLayer2_loss: 1.4702 - rightLayer2_loss: 0.8654
Epoch 00011: val_loss improved from 4.59099 to 4.58340, saving model to ./experiments/0.011999999999999997_weights.h5
22433/22433 [==============================] - 28s 1ms/step
** write log to ./experiments/0.011999999999999997_test.log **
auroc 0Effusion: 0.6624430128249766

auprc 0Effusion: 0.22407131562057594

auroc 1Effusion: 0.5804963417387929

auprc 1Effusion: 0.1390461261366621

auroc 2Effusion: 0.5601147325245572

auprc 2Effusion: 0.15124090449219071

auroc 3Effusion: 0.5033780650759136

auprc 3Effusion: 0.11341989292942219

auroc 4Effusion: 0.6681966144860302

auprc 4Effusion: 0.17878474332120337

auroc 5Effusion: 0.4351027067217512

auprc 5Effusion: 0.1021392883936755

mean auroc: 0.5682885788953369

mean auprc: 0.15145037848228832

max auroc: 0.668196614

Epoch 4/11
9260/9261 [============================>.] - ETA: 0s - loss: 4.7298 - leftLayer1_loss: 0.0736 - midLayer1_loss: 1.3627 - rightLayer1_loss: 0.9129 - leftLayer2_loss: 0.0500 - midLayer2_loss: 1.4512 - rightLayer2_loss: 0.8794
Epoch 00004: val_loss improved from 4.80135 to 4.74331, saving model to ./experiments/0.012999999999999996_weights.h5
9261/9261 [==============================] - 23s 2ms/step - loss: 4.7300 - leftLayer1_loss: 0.0736 - midLayer1_loss: 1.3627 - rightLayer1_loss: 0.9129 - leftLayer2_loss: 0.0500 - midLayer2_loss: 1.4512 - rightLayer2_loss: 0.8795 - val_loss: 4.7433 - val_leftLayer1_loss: 0.0692 - val_midLayer1_loss: 1.3579 - val_rightLayer1_loss: 0.8973 - val_leftLayer2_loss: 0.0654 - val_midLayer2_loss: 1.3866 - val_rightLayer2_loss: 0.9670
Epoch 5/11
9240/9261 [============================>.] - ETA: 0s - loss: 4.6985 - leftLayer1_loss: 0.0661 - midLayer1_loss: 1.3630 - rightLayer1_loss: 0.8978 - leftLayer2_loss: 0.0454 - midLayer2_loss: 1.4516 - rightLaye

  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 9261 steps, validate for 1292 steps
Epoch 1/11
9258/9261 [============================>.] - ETA: 0s - loss: 5.5784 - leftLayer1_loss: 0.1113 - midLayer1_loss: 1.3449 - rightLayer1_loss: 1.4275 - leftLayer2_loss: 0.1029 - midLayer2_loss: 1.4234 - rightLayer2_loss: 1.1684
Epoch 00001: val_loss improved from inf to 5.19395, saving model to ./experiments/0.013999999999999995_weights.h5
9261/9261 [==============================] - 24s 3ms/step - loss: 5.5783 - leftLayer1_loss: 0.1113 - midLayer1_loss: 1.3449 - rightLayer1_loss: 1.4274 - leftLayer2_loss: 0.1029 - midLayer2_loss: 1.4234 - rightLayer2_loss: 1.1683 - val_loss: 5.1939 - val_leftLayer1_loss: 0.1026 - val_midLayer1_loss: 1.3420 - val_rightLayer1_loss: 1.1454 - val_leftLayer2_loss: 0.0994 - val_midLayer2_loss: 1.3500 - val_rightLayer2_loss: 1

9255/9261 [============================>.] - ETA: 0s - loss: 4.5913 - leftLayer1_loss: 0.0478 - midLayer1_loss: 1.3448 - rightLayer1_loss: 0.8713 - leftLayer2_loss: 0.0374 - midLayer2_loss: 1.4250 - rightLayer2_loss: 0.8650
Epoch 00011: val_loss improved from 4.59183 to 4.58275, saving model to ./experiments/0.013999999999999995_weights.h5
22433/22433 [==============================] - 27s 1ms/step
** write log to ./experiments/0.013999999999999995_test.log **
auroc 0Effusion: 0.41544430631608265

auprc 0Effusion: 0.09688982707317102

auroc 1Effusion: 0.4192298094658927

auprc 1Effusion: 0.09637534738261386

auroc 2Effusion: 0.5902711006202934

auprc 2Effusion: 0.15166167168330538

auroc 3Effusion: 0.463000594915127

auprc 3Effusion: 0.10504698500873796

auroc 4Effusion: 0.28690159337689447

auprc 4Effusion: 0.0796332184029697

auroc 5Effusion: 0.4170123196992189

auprc 5Effusion: 0.09561146964295676

mean auroc: 0.4319766207322515

mean auprc: 0.10420308653229245

max auroc: 0.5902711

Epoch 4/11
9246/9261 [============================>.] - ETA: 0s - loss: 4.6706 - leftLayer1_loss: 0.0730 - midLayer1_loss: 1.3857 - rightLayer1_loss: 0.9083 - leftLayer2_loss: 0.0494 - midLayer2_loss: 1.3744 - rightLayer2_loss: 0.8798
Epoch 00004: val_loss improved from 4.78406 to 4.72535, saving model to ./experiments/0.014999999999999994_weights.h5
9261/9261 [==============================] - 22s 2ms/step - loss: 4.6703 - leftLayer1_loss: 0.0730 - midLayer1_loss: 1.3857 - rightLayer1_loss: 0.9082 - leftLayer2_loss: 0.0494 - midLayer2_loss: 1.3742 - rightLayer2_loss: 0.8798 - val_loss: 4.7254 - val_leftLayer1_loss: 0.0686 - val_midLayer1_loss: 1.3763 - val_rightLayer1_loss: 0.8943 - val_leftLayer2_loss: 0.0663 - val_midLayer2_loss: 1.3365 - val_rightLayer2_loss: 0.9833
Epoch 5/11
9237/9261 [============================>.] - ETA: 0s - loss: 4.6395 - leftLayer1_loss: 0.0653 - midLayer1_loss: 1.3854 - rightLayer1_loss: 0.8936 - leftLayer2_loss: 0.0450 - midLayer2_loss: 1.3746 - rightLaye

In [17]:
print(np.max(maxi))

0.8031153185829366
